### **Grand Canyon Sandbar Timelapse creator**

A script to create timelapses from sandbar imagery (testing script)

Created by Ryan E. Lima ryan.lima@nau.edu for the USGS GCMRC

#### To Do List
1.  Turn into python Package
2.  improve commenting
3.  create in-depth readme
4.  add error messages
5.  provide sample dataset

In [1]:
# load libraries
import re
import os
import cv2
import glob
from datetime import datetime



### Next we will create a function which gets a list of images from an input file directory 

In [2]:
##------------ get list of images----------------------------#
# sample image directories from sandbars for which timelapses will be made

def get_image_files(directory):
    """
    Get a list of image file paths from the given directory.
    Supports .jpg, .JPG, .jpeg, .JPEG file extensions.
    """
    extensions = ['jpg', 'JPG', 'jpeg', 'JPEG']
    files = []
    for ext in extensions:
        files.extend(glob.glob(os.path.join(directory, f'*.{ext}')))
    return files


In [4]:
#test the get_image_files() function
site_name = 'RC2255Ra'
dir_name = 'M:' + os.sep + 'Remote Cameras' + os.sep + 'full_res' + os.sep + site_name # replace this with your file path
# ensure the path name is correct
print(dir_name)
# get list of image files - run the 
image_files = get_image_files(dir_name)
len(image_files)


M:\Remote Cameras\full_res\RC2255Ra


39044

In [5]:


def parse_filename(filename):
    """
    Parse the filename to extract date and time.
    Handles filenames with optional '_web' before the extension and is case-insensitive to extensions.
    Returns a datetime object if the filename matches the pattern, otherwise None.
    """
    # Updated regex pattern to be case insensitive and to match both '.jpg' and '.jpeg'
    pattern = r'RC\d{4}[RL][a-z]?_(\d{8})_(\d{4})(_web)?\.(jpg|jpeg|JPG|JPEG)$'
    match = re.match(pattern, filename, re.IGNORECASE)
    if match:
        date_str, time_str, _, _ = match.groups()
        return datetime.strptime(date_str + time_str, '%Y%m%d%H%M')
    return None


def get_file_size(file_path):
    """
    Get and print the file size of the given image file.
    """
    size_bytes = os.path.getsize(file_path)
    size_kb = size_bytes / 1024
    print(f"File: {file_path}, Size: {size_kb:.2f} KB")
    return size_kb

def resize_image(image, scale_percent):
    """
    Resize the given image by the specified scale percentage.
    """
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)
    return cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
    
#def filter_images(folder_path, start_time, end_time):
#    """
#    Filter images in the folder based on the specified time range.
#    Returns a list of filenames that fall within the time range, sorted by datetime.
#    """
#    filtered_files = []
#    for filename in os.listdir(folder_path):
#        file_datetime = parse_filename(filename)
#        if file_datetime and start_time <= file_datetime.time() <= end_time:
#            filtered_files.append((file_datetime, os.path.join(folder_path, filename)))

    ## Sort the list by datetime
#    filtered_files.sort(key=lambda x: x[0])
#
#    # Extract just the file paths
#    sorted_files = [file[1] for file in filtered_files]
#    print(f'The Directory {folder_path} contains {len(sorted_files)} within the specified start_time, end_time range')
#    print(f'The first image is {sorted_files[0]} and the last image of the series is {sorted_files[-1]}')
#    return sorted_files
def filter_images(folder_path, start_time, end_time, start_year=None, end_year=None):
    """
    Filter images in the folder based on the specified time range and optionally by year.
    Parameters:
        folder_path (str): Path to the directory containing images.
        start_time (datetime.time): Start time to filter images.
        end_time (datetime.time): End time to filter images.
        start_year (int, optional): Start year to include images from. None means no lower year limit.
        end_year (int, optional): End year to include images until. None means no upper year limit.
    Returns:
        List of filenames that fall within the specified time and optionally year range.
    """
    filtered_files = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        file_datetime = parse_filename(filename)
        if file_datetime:
            # Check if the file's datetime falls within the specified year and time range
            if ((start_year is None or file_datetime.year >= start_year) and
                (end_year is None or file_datetime.year <= end_year) and
                start_time <= file_datetime.time() <= end_time):
                filtered_files.append((file_datetime, file_path))

    # Sort the list by datetime
    filtered_files.sort(key=lambda x: x[0])

    # Extract just the file paths
    sorted_files = [file[1] for file in filtered_files]
    return sorted_files

def put_date_on_image(img, date_str):
    """
    Puts the date string on the bottom-left corner of the image.
    """
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 2
    font_color = (255, 255, 255)  # white color
    line_type = 3

    cv2.putText(img, date_str, 
                (10, img.shape[0] - 10),  # position at bottom left
                font, 
                font_scale, 
                font_color, 
                line_type)


def create_timelapse(site_name, image_files,out_dir = os.getcwd(),frame_rate=7, resize_percent=100):
    """
    Create a timelapse video, with options to specify an out_dir,select frame rate, and resize images by percent
    default framerate = 7 frames per second or 1 week per second if images are sorted by time of day to utilize a single image
    per day, which is recommended to reduce changes in water elevation and flickering from variable lighting at different times of day
    
    """
    current_date = datetime.now().strftime('%Y%m%d')
    output_file = f'{site_name}_timelapse_{current_date}.mp4'
    output_file = os.path.join(out_dir, output_file)

    if len(image_files) == 0:
        print("No images to process.")
        return None
        # Initialize VideoWriter with the first frame's dimensions after resizing
    first_frame = cv2.imread(image_files[0])
    if resize_percent != 100:
        first_frame = resize_image(first_frame, resize_percent)
    height, width, layers = first_frame.shape
    size = (width, height)
    out = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*'mp4v'), frame_rate, size)
    #out = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*'XVID'), frame_rate, size) # if you want .avi format
    n_images = 1
    nn = len(image_files)
    for filename in image_files:
        print(f'processing image {filename} - {n_images} out of {nn}') 
        img = cv2.imread(filename)
        if resize_percent != 100:
            img = resize_image(img, resize_percent)
        date_time = parse_filename(os.path.basename(filename))
        date_str = date_time.strftime('%Y-%m-%d %H:%M')
        put_date_on_image(img, date_str)
        out.write(img)
        n_images = n_images+1
        #get_file_size(filename)

    out.release()
    vid_size = get_file_size(output_file)
    
    return output_file



In [6]:
# test the parse filename function
site_name = 'RC0235L'
dir_name = 'M:' + os.sep + 'Remote Cameras' + os.sep + 'full_res' + os.sep + site_name
print(dir_name)
image_files = get_image_files(dir_name)
n = len(image_files)
print(n)
test_filename  = image_files[n-1]
print(test_filename)
date_time = parse_filename(os.path.basename(test_filename))
print(date_time)

M:\Remote Cameras\full_res\RC0235L
31908
M:\Remote Cameras\full_res\RC0235L\RC0235L_20210717_1559.JPG
2021-07-17 15:59:00


In [12]:
list_site = ["RC0701R","RC0701Rsubsedt",
             "RC0817L","RC0846R","RC0876L","RC0917R","RC0938L","RC1044R","RC1194R",
             "RC1198R","RC1227R","RC1233Lb","RC1258R","RC1377L","RC1396R","RC1459L",
             "RC1671L","RC1722L","RC1726La","RC1726Lb","RC1833R","RC1946L","RC1995R",
             "RC2023R","RC2133L","RC2201R","RC2207L","RC2255Ra","RC2255Rb"]

for name_site in list_site:
    print(name_site)

RC0701R
RC0701Rsubsedt
RC0817L
RC0846R
RC0876L
RC0917R
RC0938L
RC1044R
RC1194R
RC1198R
RC1227R
RC1233Lb
RC1258R
RC1377L
RC1396R
RC1459L
RC1671L
RC1722L
RC1726La
RC1726Lb
RC1833R
RC1946L
RC1995R
RC2023R
RC2133L
RC2201R
RC2207L
RC2255Ra
RC2255Rb


In [ ]:
# Example usage

for name_site in list_site:
    site_name = name_site # change this to the target site, assuming images are stored in directory of the same name
    path_name = 'M:/Remote Cameras/full_res' + os.sep + site_name
    print(site_name)

    folder_path = path_name
    
    
    '''
    Due to variable lighting in the canyon throughout the day, and the variable water levels,
    its advisable to limit the timelapse to a single image per day rather than 5 per day. utlizing
    the start_time and end_time arguments. All times in MST military time 00:00 - 24:59 %H:%M
    
    If not using web-res images, is suggested to re-size images by at least 25% to prevent very large video files
    '''
    start_time = datetime.strptime('10:00', '%H:%M').time()
    end_time = datetime.strptime('13:00', '%H:%M').time()
    
    resize_percent = 25  # Resize images to 50% of their original size
    print('filtering images')
    filtered_files = filter_images(folder_path, start_time, end_time)
    n_images = len(filtered_files)
    print(n_images)
    print('Images filtered')
    
    print('creating timelapse')
    #filtered_files = filtered_files[0:100] # to test script on subset of data, use this line, otherwise comment out for full timelapse
    video_filename = create_timelapse(site_name, filtered_files,out_dir = os.getcwd(), frame_rate=7, resize_percent=resize_percent)
    if video_filename:
        print(f"Timelapse video created: {video_filename}")
    print(site_name + " : Time lapse complete" )

RC0701R
filtering images
1832
Images filtered
creating timelapse
processing image M:/Remote Cameras/full_res\RC0701R\RC0701R_20130929_1157.JPG - 1 out of 1832
processing image M:/Remote Cameras/full_res\RC0701R\RC0701R_20130930_1157.JPG - 2 out of 1832
processing image M:/Remote Cameras/full_res\RC0701R\RC0701R_20131001_1157.JPG - 3 out of 1832
processing image M:/Remote Cameras/full_res\RC0701R\RC0701R_20131002_1157.JPG - 4 out of 1832
processing image M:/Remote Cameras/full_res\RC0701R\RC0701R_20131003_1157.JPG - 5 out of 1832
processing image M:/Remote Cameras/full_res\RC0701R\RC0701R_20131004_1157.JPG - 6 out of 1832
processing image M:/Remote Cameras/full_res\RC0701R\RC0701R_20131005_1157.JPG - 7 out of 1832
processing image M:/Remote Cameras/full_res\RC0701R\RC0701R_20131006_1157.JPG - 8 out of 1832
processing image M:/Remote Cameras/full_res\RC0701R\RC0701R_20131007_1157.JPG - 9 out of 1832
processing image M:/Remote Cameras/full_res\RC0701R\RC0701R_20131008_1157.JPG - 10 out of

In [8]:
#video_filename = create_timelapse(site_name, filtered_files,out_dir = os.getcwd(), frame_rate=7, resize_percent=resize_percent)
#if video_filename:
#    print(f"Timelapse video created: {video_filename}")

processing image M:/Remote Cameras/full_res\RC0450L\RC0450L_20080304_1156.JPG - 1 out of 4853
processing image M:/Remote Cameras/full_res\RC0450L\RC0450L_20080305_1156.JPG - 2 out of 4853
processing image M:/Remote Cameras/full_res\RC0450L\RC0450L_20080306_1156.JPG - 3 out of 4853
processing image M:/Remote Cameras/full_res\RC0450L\RC0450L_20080307_1156.JPG - 4 out of 4853
processing image M:/Remote Cameras/full_res\RC0450L\RC0450L_20080308_1156.JPG - 5 out of 4853
processing image M:/Remote Cameras/full_res\RC0450L\RC0450L_20080309_1156.JPG - 6 out of 4853
processing image M:/Remote Cameras/full_res\RC0450L\RC0450L_20080310_1156.JPG - 7 out of 4853
processing image M:/Remote Cameras/full_res\RC0450L\RC0450L_20080311_1156.JPG - 8 out of 4853
processing image M:/Remote Cameras/full_res\RC0450L\RC0450L_20080312_1156.JPG - 9 out of 4853
processing image M:/Remote Cameras/full_res\RC0450L\RC0450L_20080313_1156.JPG - 10 out of 4853
processing image M:/Remote Cameras/full_res\RC0450L\RC0450L

In [9]:
#print(site_name + " : Time lapse complete" )

RC0450L : Time lapse complete
